## What's on the [`xtensor`](https://github.com/psi4/psi4/pull/1443) pull request?

Quite some template trickery:

1. substitution failure is not an error ([SFINAE](https://en.cppreference.com/w/cpp/language/sfinae)) with `std::enable_if` and type traits. Allows arbitrary types, excluding corner cases.
2. curiously recurring template pattern ([CRTP](https://en.wikipedia.org/wiki/Curiously_recurring_template_pattern)). Allows arbitrary rank with minimal code duplication.

### Under `psi4/src/psi4/libmints`

1. The C++ data structure `tensor.h`
2. Implementation details for rank-1 (vector) `tensor_impl_vector.h`. This is called `Vector_<T>`.
3. Implementation details for rank-2 (matrix) `tensor_impl_matrix.h`. This is called `Matrix_<T>`.
3. Implementation details for rank-N `tensor_impl.h`
4. 3D vectors are in `vector3.h`
3. Builders, operators, linear algebra `linalg.h`

### Under `psi4/src`

Linear algebra is a _submodule_ of `psi4.core`:
1. The Python bindings `export_linalg.cc`

### Under `psi4/linalg`

1. Python helpers `utils.py`
2. Python factory-like interface for builders `factory.py`
3. `__init__.py` exports the `psi4.core.linalg` submodule as `psi4.linalg`

# Using the `xtensor` branch

## Importing

In [ ]:
import psi4
import numpy as np

help(psi4.linalg)

## Building vectors and matrices

The implemented types are:

* C++ `float` (single-precision real number) which maps to `np.float32`
* C++ `double` (double-precision real numbers) which maps to `np.float64`
* C++ `std::complex<double>` (double precision complex number) which maps to `np.complex128`

#### Caveats/limitations

- **Other types are possible**, but one needs to be careful with **mixing** them. Some operations are **NOT** defined in the C++ standards, _e.g._ summing a `float` and `std::complex<float>`
- Compiling with xsimd is currently **broken** if the complex number bindings are included.

### Using the constructors

The type information needs to be given **up front**. For a labeled, blocked vector of double-precision numbers we'd do:

In [8]:
dimpi = psi4.core.Dimension([3, 2, 1, 0])
# labeled, 4-irrep vector
v1 = psi4.linalg.Vector_D(label="v1", dimpi=dimpi, fill_value=3.14)
print(v1)

  ## v1 Vector<double> (Symmetry 0) ##

  Irrep: 1 Shape: {3}
{ 3.14,  3.14,  3.14}

  Irrep: 2 Shape: {2}
{ 3.14,  3.14}

  Irrep: 3 Shape: {1}
{ 3.14}

  Irrep: 4 Shape: {0}
    (empty)




and similarly for a matrix:

In [18]:
rowspi = psi4.core.Dimension([3, 2, 1, 4])
colspi = psi4.core.Dimension([4, 2, 0, 2])
# labeled, 4-irrep matrix
m1 = psi4.linalg.Matrix_CD(label='m1', rowspi=rowspi, colspi=colspi, fill_value=(1.0-1.0j))
print(m1)

  ## m1 Matrix<complex<double>> (Symmetry 0) ##

  Irrep: 1 Shape: {3, 4}
{{ 1.-1.i,  1.-1.i,  1.-1.i,  1.-1.i},
 { 1.-1.i,  1.-1.i,  1.-1.i,  1.-1.i},
 { 1.-1.i,  1.-1.i,  1.-1.i,  1.-1.i}}

  Irrep: 2 Shape: {2, 2}
{{ 1.-1.i,  1.-1.i},
 { 1.-1.i,  1.-1.i}}

  Irrep: 3 Shape: {1, 0}
    (empty)

  Irrep: 4 Shape: {4, 2}
{{ 1.-1.i,  1.-1.i},
 { 1.-1.i,  1.-1.i},
 { 1.-1.i,  1.-1.i},
 { 1.-1.i,  1.-1.i}}




### Using the factory

This is slightly more Pythonic:
* `kwargs` are the same as the constructors
* the type information is passed as **an argument**: `dtype=np.float32`

In [22]:
v1_f = psi4.linalg.Vector_(dimpi=dimpi, fill_value=3.14, label="v1_f", dtype=np.float32)
print(v1_f)

  ## v1_f Vector<float> (Symmetry 0) ##

  Irrep: 1 Shape: {3}
{ 3.14000010490417,  3.14000010490417,  3.14000010490417}

  Irrep: 2 Shape: {2}
{ 3.14000010490417,  3.14000010490417}

  Irrep: 3 Shape: {1}
{ 3.14000010490417}

  Irrep: 4 Shape: {0}
    (empty)




In [23]:
m1_d = psi4.linalg.Matrix_(fill_value=-0.5, label="m1_d", rowspi=rowspi, colspi=colspi, dtype=np.float64)
print(m1_d)

  ## m1_d Matrix<double> (Symmetry 0) ##

  Irrep: 1 Shape: {3, 4}
{{-0.5, -0.5, -0.5, -0.5},
 {-0.5, -0.5, -0.5, -0.5},
 {-0.5, -0.5, -0.5, -0.5}}

  Irrep: 2 Shape: {2, 2}
{{-0.5, -0.5},
 {-0.5, -0.5}}

  Irrep: 3 Shape: {1, 0}
    (empty)

  Irrep: 4 Shape: {4, 2}
{{-0.5, -0.5},
 {-0.5, -0.5},
 {-0.5, -0.5},
 {-0.5, -0.5}}




### Using the builders

This is "just" like NumPy. Currently available builders:

* `zeros_like`
* `ones_like`
* `full_like`

**Caveats/limitations** Mixing types is **NOT** possible with the builders, _e.g._ creating a complex object with the shape of single-precision one won't work.

Transmuting builders:

* from `Vector`: `auto v_ = transmute(v);`
* from `Matrix`: `auto m_ = transmute(m);`
* **to be implemented** from `dpdfile2`
* **to be implemented** from `dpdfile4`

These will be deprecated in the future.

Most of the above is tested in `tests/pytests/test_tensor_ctors.py`

## Linear algebra

`xtensor` and `xtensor-blas` come with all that is needed. It's a matter of "lifting" to the blocked structure, _i.e._ looping over blocks. Implemented so far:

* `cholesky`. Tested in `tests/pytests/test_tensor_cholesky.py`
* `qr`
* `eigvals`. Tested in `tests/pytests/test_tensor_eig.py`
* `eig`.  Tested in `tests/pytests/test_tensor_eig.py`
* `eigvalsh`. Tested in `tests/pytests/test_tensor_eigh.py`
* `eigh`. Tested in `tests/pytests/test_tensor_eigh.py`
* `svd`
* `doublet`. Tested in `tests/pytests/test_tensor_2d_doublet.py`

## NumPy interoperability

The blocks of each tensor have `numpy.ndarray` type. So you can manipulate them directly with NumPy.

In [33]:
print(type(v1))
print(type(v1[0]))

print(type(v1_f))
print(type(v1_f[0]))

print(type(m1))
print(type(m1[0]))

print(type(m1_d))
print(type(m1_d[0]))

for h, blk in enumerate(m1_d):
    print(f"At irrep {h}, the block is\n {blk}")

<class 'psi4.core.linalg.Vector_D'>
<class 'numpy.ndarray'>
<class 'psi4.core.linalg.Vector_F'>
<class 'numpy.ndarray'>
<class 'psi4.core.linalg.Matrix_CD'>
<class 'numpy.ndarray'>
<class 'psi4.core.linalg.Matrix_D'>
<class 'numpy.ndarray'>
At irrep 0, the block is
 [[-0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5]]
At irrep 1, the block is
 [[-0.5 -0.5]
 [-0.5 -0.5]]
At irrep 2, the block is
 []
At irrep 3, the block is
 [[-0.5 -0.5]
 [-0.5 -0.5]
 [-0.5 -0.5]
 [-0.5 -0.5]]


In [40]:
int_row = 10
int_col = 20

# unlabeled 1-irrep matrix
m2 = psi4.linalg.Matrix_(rows=int_row, cols=int_col)
np_m2 = np.arange(int_row * int_col).reshape(int_row, int_col)
m2[0] = np.arange(int_row * int_col).reshape(int_row, int_col)
assert psi4.compare_values(np_m2, m2[0], "Assign NumPy array to unlabeled, 1-irrep matrix")

dim_row = psi4.core.Dimension([3, 2, 1, 4])
dim_col = psi4.core.Dimension([4, 2, 0, 2])
# labeled, 4-irrep matrix
m3 = psi4.linalg.Matrix_(label='m3', rowspi=dim_row, colspi=dim_col)
# Get block, then modify it
block = m3[3]
block[:] = np.arange(m3[3].size).reshape(*m3[3].shape)
assert psi4.compare_values(np.arange(m3[3].size).reshape(*m3[3].shape), m3[3],
        "Assign NumPy array to block extracted from labeled, 4-irrep matrix")

# Assign to block
m3[1] = np.arange(m3[1].size).reshape(*m3[1].shape)
assert psi4.compare_values(np.arange(m3[1].size).reshape(*m3[1].shape), m3[1],
        "Assign NumPy array to a block in labeled, 4-irrep matrix")

    Assign NumPy array to unlabeled, 1-irrep matrix...................PASSED
    Assign NumPy array to block extracted from labeled, 4-irrep matrixPASSED
    Assign NumPy array to a block in labeled, 4-irrep matrix..........PASSED


In [46]:
# Make a Hermitian blocked matrix (see also psi4/linalg/utils.py)
dim = psi4.core.Dimension([2, 3, 4, 5, 6, 7, 8, 9])
m = psi4.linalg.Matrix_(label='test', rowspi=dim, colspi=dim, dtype=np.complex128)

for h in range(m.nirrep):
    shape = (m.rows(h), m.cols(h ^ m.symmetry))
    blk = np.random.randn(*shape) + np.random.randn(*shape) * 1j
    U, s, V = np.linalg.svd(np.dot(blk.conj().T, blk))
    m[h] = np.dot(np.dot(U, 1.0 + np.diag(np.random.rand(shape[0]))), V)
    
res = psi4.linalg.cholesky(m)

expected = [np.linalg.cholesky(blk) for blk in m]

for blk_idx, blks in enumerate(zip(expected, res)):
    assert psi4.compare_values(blks[0], blks[1], f"Cholesky factor L for block[{blk_idx}]")

    Cholesky factor L for block[0]....................................PASSED
    Cholesky factor L for block[1]....................................PASSED
    Cholesky factor L for block[2]....................................PASSED
    Cholesky factor L for block[3]....................................PASSED
    Cholesky factor L for block[4]....................................PASSED
    Cholesky factor L for block[5]....................................PASSED
    Cholesky factor L for block[6]....................................PASSED
    Cholesky factor L for block[7]....................................PASSED


## **All instances of `Vector` and `Matrix` need to be replaced with `Vector_<double>` and `Matrix_<double>`.**

## The transition so far

Done so far:
* `Vector3` implementation replaced everywhere.
* `Vector_<double>` replacement in files related to `export_functional.cc`.
* `Vector_<double>` replacement in files related to `export_fock.cc`.

## **Proposal to move forward**

It is **unrealistic** that I'll be able to move all of Psi4 over.

1. The current work gets merged (after review!) to a branch (_e.g._ `xtensor`) under `psi4/psi4`.
2. Work gets split among volunteers folder-wise, PRs go to `psi4/psi4:xtensor`.
3. First replace `Vector` with `Vector_<double>`. Fix compiler errors:
   * Change signatures (most of the time sufficient).
   * Implement functions that do not exist yet.
   * Rewrite some of the calling code.
4. Run relevant tests. Ideally no functionality should break.